# Internal Scripts

In [1]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/32/ab/22530cc1b2114e6067eece94a333d6c749fa1c56a009f0721e51c181ea53/Django-2.1.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/02/bae88c4aaea4256d890adbf3f7cf33e59a443f9985cf91cd08a35656676a/numpy-1.15.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e1/d8/feeb346d41f181e83fba45224ab14a8d8af019b48af742e047f3845d8cff/pandas-0.23.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/30/4e/27c34b62430286c6d59177a0842ed90dc789ce5d1ed740887653b898779a/pytz-2018.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/07/e6/e95c4eec6221bfd8528bcc4ea252a850bffcc4be88ebc367e23a1a84b0bb/xlrd-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cf/37/830a21a2f6fa3e4cbda743c34fc94bee2ac07cb3ccf1630069fa8bc3190b/XlsxWriter-1.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.

In [14]:
from xlwt import Workbook
from xlsxwriter import Workbook
from zipfile import ZipFile
from urllib.parse import urlparse, parse_qs
import pandas as pd
import xlrd
import os
import glob
import shutil


In [24]:
# DIR = os.listdir('Input_Files')
loc = '/home/wsk/code/company/absentiaVR/notebook-avr/Scripto/Input_Files/navratri-ploan-15to23oct.xlsx'
# http://localhost:8888/edit/Scripto/Input_Files/durga-ploan-15to23oct.xlsx
# http://localhost:8888/edit/Scripto/Input_Files/navratri-bloan-15to23oct.xlsx
# http://localhost:8888/edit/Scripto/Input_Files/navratri-ploan-15to23oct.xlsx

In [25]:
df = pd.read_excel(loc)
a = list(df.columns)

all_urls = [_a for _a in a if 'url (actionDetails' in _a]
all_times = [_a for _a in a if 'serverTimePretty (actionDetails' in _a]

visitType = []
visitTyped = {}
for i in df['visitIp']:
    if i not in visitTyped.keys():
        visitTyped[i] = 1
        visitType.append('Unique')
    else:
        visitType.append('Return')

pagesCount = []
vcf = []
redirected = []
total_redirected = []
outlink = []
t_time = []
videoURL = []
videoURL_count = []
for _r in range(df.shape[0]):
    c_vcf = 0
    pagescount = 0
    redirectedcount = 0
    outlink_url = []
    tm = []
    video_url = []
    video_url_count = 0
    for _j in all_urls:
        if pd.notna(df[_j][_r]) and '.vcf' in df[_j][_r]:
            c_vcf = 1
        if pd.notna(df[_j][_r]):
            pagescount += 1
        if pd.notna(df[_j][_r]) and 'hdfc-navratri-ploan.pictorpv.com' in df[_j][_r]:
            video_url_count += 1
            video_url = df[_j][_r]
            
        if pd.notna(df[_j][_r]) and 'https://leads.hdfcbank.com/applications/webforms/' in df[_j][_r]:
            num = int(_j.split()[2][0])
            tim = df['serverTimePretty (actionDetails %d)'% num][_r].split()[-1]
            tm.append(tim)
            redirectedcount += 1
            outlink_url = df[_j][_r]
        else:
            tm.append('')
            
    if video_url == []:
        videoURL.append('')
    else:
        videoURL.append(video_url)
    videoURL_count.append(video_url_count)
            
    if c_vcf == 0:
        vcf.append('No')
    else:
        vcf.append('Yes')
    if redirectedcount == 0:
        redirected.append('No')
    else:
        redirected.append('Yes')
    if outlink_url == []:
        outlink.append('')
    else:
        outlink.append(outlink_url)
        
    t_time.append(tm)
    pagesCount.append(pagescount)
    total_redirected.append(redirectedcount)

for w1 in range(len(t_time[0])):
    li = []
    for w2 in range(len(t_time)):
        li.append(t_time[w2][w1])
    df['OutlinkTime%s'% w1] = li

custID = []
custName = []
for k in outlink:
    if k!='':
        parsed = urlparse(k)
        id = parse_qs(parsed.query)['tbcustid']
        name = parse_qs(parsed.query)['tbcustname']
        custID.append(id[0])
        custName.append(name[0])
    else:
        custID.append('')
        custName.append('')
df['video url'] = videoURL
df['video url count'] = videoURL_count
df['visitType'] = visitType
df['pagesCount'] = pagesCount
df['redirected'] = redirected
df['TotalRedirected'] = total_redirected
df['outlink'] = outlink
df['custid'] = custID
df['custname'] = custName
df['vcf downloaded'] = vcf


times = ['OutlinkTime%s'%_a for _a in range(len(t_time[0]))]
remap = ['visitType', 'visitIp', 'pagesCount', 'serverDatePretty', 'serverTimePretty',
         'visitDurationPretty', 'redirected', 'TotalRedirected', 'outlink', 'custid', 'custname', 'vcf downloaded', 'video url', 'video url count']
remap += times

print(len(videoURL_count), videoURL_count)
print(len(videoURL), videoURL)
print(df['video url'])


456 [1, 5, 4, 4, 1, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 3, 1, 1, 1, 1, 1, 1, 4, 1, 6, 1, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 4, 1, 1, 1, 1, 6, 8, 1, 1, 1, 3, 5, 1, 1, 3, 1, 1, 1, 1, 6, 5, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 4, 1, 7, 4, 1, 1, 0, 2, 2, 1, 3, 7, 9, 6, 1, 18, 2, 1, 1, 1, 2, 6, 1, 2, 8, 1, 4, 1, 1, 5, 1, 0, 4, 1, 1, 3, 1, 1, 1, 2, 2, 4, 2, 5, 3, 1, 1, 2, 0, 1, 4, 1, 1, 3, 1, 1, 1, 13, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 4, 1, 1, 3, 1, 1, 1, 5, 1, 15, 3, 1, 10, 7, 1, 4, 0, 1, 6, 1, 1, 1, 2, 1, 1, 5, 16, 5, 10, 1, 1, 1, 1, 1, 5, 1, 1, 4, 1, 1, 1, 6, 14, 3, 1, 1, 1, 1, 14, 9, 15, 1, 3, 1, 1, 1, 9, 1, 5, 1, 9, 1, 6, 1, 1, 1, 1, 4, 1, 1, 4, 1, 4, 4, 1, 11, 1, 5, 6, 14, 2, 8, 4, 27, 2, 4, 8, 3, 1, 2, 8, 1, 1, 4, 1, 1, 1, 6, 1, 9, 10, 1, 5, 5, 1, 1, 1, 6, 6, 38, 4, 4, 4, 2, 4, 3, 1, 4, 3, 9, 3, 1, 12, 1, 1, 7, 1, 1, 6, 1, 10, 2, 16, 1, 1, 11, 2, 10, 2, 8, 1, 4, 1, 10, 4, 4, 1, 3, 4, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 32, 5, 1, 10, 3, 1, 17, 1, 12, 4, 4, 1, 6, 1, 1, 4, 1, 5, 3, 6,

In [26]:
writer = pd.ExcelWriter('Output_Files/navratri-ploan-15to23oct.xlsx', engine='xlsxwriter', options={'strings_to_urls': False})
df.to_excel(writer, index=False, columns=remap)
writer.close()